## Configurações iniciais

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account
import polars as pl
import pandas as pd
import os
import gc

In [ ]:
pasta_projeto = "D:\\__case_ifood"

# Caminho do arquivo JSON da conta de serviço
credencial_gcp = os.path.join(pasta_projeto,"case-ifood-fsg-6f1d7cf34e08.json")

# Autenticando
credencial = service_account.Credentials.from_service_account_file(credencial_gcp)

# Cliente BigQuery
client = bigquery.Client(credentials=credencial, project="case-ifood-fsg")


### Função para Criar o dataset e inserir a tabela no BQ

In [ ]:
def split_dataframe(df, chunk_size):
    """Divide um DataFrame Pandas em pedaços menores."""
    return [df[i:i+chunk_size] for i in range(0, len(df), chunk_size)]

def criar_dataset_e_tabela_append(df, dataset_nome, tabela_nome, client, project_id, location="southamerica-east1", chunk_size=1_000_000):
    """
    Cria um dataset (se necessário) e insere os dados em uma tabela BigQuery usando chunks.
    Sempre faz append (WRITE_APPEND). Suporta DataFrames do Pandas ou Polars.
    """

    dataset_id = f"{project_id}.{dataset_nome}"
    table_id = f"{dataset_id}.{tabela_nome}"

    # 1. Criar dataset se não existir
    try:
        dataset = bigquery.Dataset(dataset_id)
        dataset.location = location
        client.create_dataset(dataset, exists_ok=True)
        print(f"Dataset '{dataset_id}' pronto.")
    except Exception as e:
        print(f"Erro ao criar dataset: {e}")
        return

    # 2. Converter de Polars para Pandas, se necessário
    if isinstance(df, pl.DataFrame):
        df = df.to_pandas()
        print("Convertido de Polars para Pandas.")

    if not isinstance(df, pd.DataFrame):
        print("O objeto fornecido não é um DataFrame válido.")
        return

    # 3. Inserir em chunks (sempre com WRITE_APPEND)
    try:
        chunks = split_dataframe(df, chunk_size)
        total_rows = 0

        for i, chunk in enumerate(chunks):
            print(f"Enviando chunk {i+1}/{len(chunks)} com {len(chunk)} linhas...")
            job_config = bigquery.LoadJobConfig(write_disposition=bigquery.WriteDisposition.WRITE_APPEND)
            job = client.load_table_from_dataframe(chunk, table_id, job_config=job_config)
            job.result()
            total_rows += len(chunk)

        print(f"Tabela '{table_id}' atualizada com {total_rows} linhas (append).")
    except Exception as e:
        print(f"Erro ao carregar tabela por chunks: {e}")


### Função para enviar os arquivos Parquet para o BQ

In [ ]:
def enviar_parquets_para_bigquery(pasta_projeto, dataset_nome, tabela_nome, client, project_id, location="southamerica-east1"):
    """
    Percorre os arquivos 'sales_*.parquet' na pasta_projeto e faz append no BigQuery.
    """
    arquivos = sorted([arq for arq in os.listdir(pasta_projeto) if arq.startswith("sales_") and arq.endswith(".parquet")])

    if not arquivos:
        print("Nenhum arquivo 'sales_*.parquet' encontrado na pasta.")
        return

    for arquivo in arquivos:
        caminho_arquivo = os.path.join(pasta_projeto, arquivo)
        print(f"Lendo arquivo: {arquivo}")

        try:
            df = pl.read_parquet(caminho_arquivo)
        except Exception as e:
            print(f"Erro ao ler {arquivo}: {e}")
            continue

        print(f"Enviando para BigQuery: {arquivo}")
        criar_dataset_e_tabela_append(
            df=df,
            dataset_nome=dataset_nome,
            tabela_nome=tabela_nome,
            client=client,
            project_id=project_id
        )

#### Filtro para otimizar o download da tabela no BQ

In [ ]:
semanas = [
    "2018-12-02", "2018-12-09", "2018-12-16", "2018-12-23",
    "2018-12-30", "2019-01-06", "2019-01-13", "2019-01-20", "2019-01-27"
]

In [ ]:
# Realizando o download da tabela e salvando em diversos arquivos parquet

# df_sales_chunk = []

for semana in semanas:
    print(f"Lendo semana: {semana}")

    qry = f"""
        WITH tabela AS (
            SELECT 
                o.order_id,
                o.order_created_at,
                od.name AS product,
                od.quantity,
                od.unitPrice,
                od.addition,
                od.discount,
                o.order_total_amount,
                od.type AS product_type,
                od.sequence,
                o.customer_id,
                o.cpf,
                COALESCE(c.customer_name, o.customer_name) AS customer_name,
                c.created_at AS customer_created_at,
                c.active AS customer_active,
                CONCAT(c.customer_phone_area,'-',c.customer_phone_number) AS customer_phone,
                ab.is_target,
                c.language AS customer_language,
                o.delivery_address_district,
                o.delivery_address_city,
                o.delivery_address_state,
                o.delivery_address_country,
                o.merchant_id,
                m.created_at AS merchant_created_at,
                m.enabled AS merchant_enabled,
                m.price_range,
                m.average_ticket,
                m.takeout_time,
                m.delivery_time,
                m.minimum_order_value,
                m.merchant_city,
                m.merchant_state,
                IF(od.order_id IS NULL, FALSE, TRUE) has_details,
                o.order_scheduled,
                o.order_scheduled_date,
                o.origin_platform,
                CAST(DATE(DATE_TRUNC(o.order_created_at,week)) AS STRING) AS semana 
            FROM silver.order o
            LEFT JOIN silver.consumer c 
                ON o.customer_id = c.customer_id
            LEFT JOIN silver.merchants m 
                ON o.merchant_id = m.id
            LEFT JOIN silver.ab_test ab 
                ON o.customer_id = ab.customer_id
            LEFT JOIN silver.order_details od 
                ON  o.order_id = od.order_id 
                AND o.cpf = od.cpf
        )
        SELECT * EXCEPT(semana)
        FROM tabela
        WHERE semana = '{semana}'
    """

    result = client.query(qry)
    arrow_table = result.to_arrow()
    df = pl.from_arrow(arrow_table)

    # Salva parquet na pasta do projeto
    nome_arquivo = f"sales_{semana}.parquet"
    caminho_arquivo = os.path.join(pasta_projeto, nome_arquivo)
    df.write_parquet(caminho_arquivo)

    print(f"Arquivo salvo: {caminho_arquivo}")

In [ ]:
del df
del arrow_table
gc.collect()

In [ ]:
# Parametros necessários para enviar os arquivos Parquet para o BigQuery
dataset_nome = "gold"
tabela_nome = "sales"
project_id = "case-ifood-fsg"

# Inserindo o DataFrame no BigQuery
enviar_parquets_para_bigquery(pasta_projeto, dataset_nome, tabela_nome, client, project_id)